In [2]:
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import pandas as pd
from random import random
import json, os.path, pickle
import math

In [24]:
# load in the two datasets that define the results
#result = pd.read_csv(f"Version1.csv") ### This was the original numbers shown
result = pd.read_csv(f"test10a.1.csv")
pairs = pd.read_csv(f"downstairs_pairs.csv")
pairs['comparision'] = pairs['pair']
result['expected_n_attempts'] = result['students_per_group']*result['attempts_per_student']*result['trials']*2
result['trial_size'] = result['students_per_group']+result['attempts_per_student']/100
pre ="geeBIs" # test prefix 
power = .8 # %
tests = [c[:-4] for c in result.columns if c.endswith('vsig')]
for t in tests:
    result[f'{t}powered'] = (result[f'{t}wrong']+result[f'{t}ns']) / (result[f'{t}sig']+result[f'{t}vsig']) < (1-power)/power
result[f'powered'] = result[f'{pre}powered'] # make a generic col

In [25]:
# A bit more quick QA on the data
fields = ['student_set', 'comparision', 'students_per_group',
       'attempts_per_student',
        'total_attempts', 'trials', 'expected_n_attempts',
          'groupA', 'groupB']
query = "expected_n_attempts != total_attempts"
bad = result.query(query)[fields]
if len(bad):
    print('size errors (ie from running on a sym against a set that does not contain enough rows!')
    display(bad)
else:
    print('no size errors')

no size errors


In [26]:
print("These are the columns of our datasets:")
print(f"pairs.columns: {list(pairs.columns)}")
print("result.columns:", list(result.columns))
print("result.columns:", )

These are the columns of our datasets:
pairs.columns: ['Unnamed: 0', 'student_set', 'pair', 'ga', 'gb', 'a', 'b', 'dif', 'comparision']
result.columns: ['Unnamed: 0', 'student_set', 'comparision', 'students_per_group', 'attempts_per_student', 'groupA', 'groupB', 'total_attempts', 'trials', 'mlmwrong', 'mlmns', 'mlmsig', 'mlmvsig', 'mlmp', 'mlmswrong', 'mlmsns', 'mlmssig', 'mlmsvsig', 'mlmsp', 'geeBIswrong', 'geeBIsns', 'geeBIssig', 'geeBIsvsig', 'geeBIsp', 'geeBIwrong', 'geeBIns', 'geeBIsig', 'geeBIvsig', 'geeBIp', 'geewrong', 'geens', 'geesig', 'geevsig', 'geep', 'query', 'time', 'key', 'expected_n_attempts', 'trial_size', 'mlmpowered', 'mlmspowered', 'geeBIspowered', 'geeBIpowered', 'geepowered', 'powered']
result.columns:


In [37]:
tests = [c[:-4] for c in result.columns if c.endswith('vsig')]
ss = result['student_set'].unique()
cs = result['comparision'].unique()
print(f"In these student pupulations: {ss}, ")
print(f"we performed these tests {tests} on these pair comparisions : {cs}")
print("These student group sizes", list(result['students_per_group'].unique()), " and these attempt counts:", list(result['attempts_per_student'].unique()))



In these student pupulations: ['variable' 'uniform' 'initial' 'second'], 
we performed these tests ['mlm', 'mlms', 'geeBIs', 'geeBI', 'gee'] on these pair comparisions : ['none' 'small' 'mod' 'mod1' 'big']
These student group sizes [30, 75, 100]  and these attempt counts: [10]


100.1

In [28]:
result[[f"{t}powered" for t in tests]]

,mlmpowered,mlmspowered,geeBIspowered,geeBIpowered,geepowered
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
5,False,False,False,False,False
6,False,False,False,False,False
7,True,True,True,True,True
8,True,True,True,True,True
9,False,False,False,False,False


In [18]:
# do some quick QA
print("All pairs found. This is good." if set(cs).issubset(pairs['pair'].unique()) else "Pairs dont match. This is bad.")

All pairs found. This is good.


In [19]:
print("Characteristics of various combinations that were tested ('pair' is aka 'comparision')")
pairs

Characteristics of various combinations that were tested ('pair' is aka 'comparision')


,Unnamed: 0,student_set,pair,ga,gb,a,b,dif,comparision
0,0,variable,none,control,nodif,58.103500,58.197500,0.094000,none
1,1,variable,vsmall,control,interventionA,58.103500,61.784375,3.680875,vsmall
2,2,variable,small,control,interventionB,58.103500,64.887875,6.784375,small
3,3,variable,mod,control,interventionBa,58.103500,70.061875,11.958375,mod
4,4,variable,mod1,control,interventionBb,58.103500,72.029125,13.925625,mod1
5,5,variable,big,control,interventionC,58.103500,73.754125,15.650625,big
6,6,variable,huge,control,interventionD,58.103500,76.755250,18.651750,huge
7,7,variable,upper,interventionC,interventionD,73.754125,76.755250,3.001125,upper
8,8,uniform,none,control,nodif,49.964000,49.975875,0.011875,none
9,9,uniform,vsmall,control,interventionA,49.964000,54.526875,4.562875,vsmall


In [20]:
print("This is the outcome of all the simulations")
result

This is the outcome of all the simulations


,Unnamed: 0,student_set,comparision,students_per_group,attempts_per_student,groupA,groupB,total_attempts,trials,mlmwrong,...,geesig,geevsig,geep,query,time,key,expected_n_attempts,trial_size,geeBIspowered,powered
0,0,variable,none,30,10,control,nodif,60000,100,3,...,1,1,"[0.2997164820412638, 0.4009229096797746, 0.291...","group == ['control', 'nodif'] and pint < 30 an...",1.627460e+12,e.100none.30.10,60000,30.1,False,False
1,1,variable,none,75,10,control,nodif,150000,100,4,...,2,1,"[0.47902692379625056, 0.29128714670078515, 0.9...","group == ['control', 'nodif'] and pint < 75 an...",1.627455e+12,e.100none.75.10,150000,75.1,False,False
2,2,variable,none,100,10,control,nodif,200000,100,4,...,2,1,"[0.6275357565107, 0.0836194140286644, 0.506513...","group == ['control', 'nodif'] and pint < 100 a...",1.627460e+12,e.100none.100.10,200000,100.1,False,False
3,3,variable,small,30,10,control,interventionB,60000,100,0,...,12,7,"[0.0411827683221373, 0.26232648935448777, 0.81...","group == ['control', 'interventionB'] and pint...",1.627460e+12,e.100small.30.10,60000,30.1,False,False
4,4,variable,small,75,10,control,interventionB,150000,100,0,...,23,20,"[0.11412910249816964, 0.04541237130433988, 0.3...","group == ['control', 'interventionB'] and pint...",1.627455e+12,e.100small.75.10,150000,75.1,False,False
5,5,variable,small,100,10,control,interventionB,200000,100,0,...,29,24,"[0.004430055430816972, 0.02519979151654244, 0....","group == ['control', 'interventionB'] and pint...",1.627460e+12,e.100small.100.10,200000,100.1,False,False
6,6,variable,mod,30,10,control,interventionBa,60000,100,0,...,26,22,"[0.1189044690187786, 0.05520109076850046, 0.00...","group == ['control', 'interventionBa'] and pin...",1.627460e+12,e.100mod.30.10,60000,30.1,False,False
7,7,variable,mod,75,10,control,interventionBa,150000,100,0,...,14,75,"[0.0017618685877294463, 0.0032433260993213146,...","group == ['control', 'interventionBa'] and pin...",1.627456e+12,e.100mod.75.10,150000,75.1,True,True
8,8,variable,mod,100,10,control,interventionBa,200000,100,0,...,9,89,"[4.5483028922637974e-06, 0.0004985321813098873...","group == ['control', 'interventionBa'] and pin...",1.627461e+12,e.100mod.100.10,200000,100.1,True,True
9,9,variable,mod1,30,10,control,interventionBb,60000,100,0,...,29,43,"[0.005981570233602056, 0.29764561866909256, 0....","group == ['control', 'interventionBb'] and pin...",1.627461e+12,e.100mod1.30.10,60000,30.1,False,False


In [38]:
# Summarise the data into a nice table:
def unpack_trial_size(s,c,test, missing):
    qs = f"student_set == '{s}' and comparision == '{c}'"
    packed = result.query(f"{qs} and {test}powered")['trial_size'].min()
    x = missing if math.isnan(packed) else packed
    size = f"{'> ' if math.isnan(packed) else ''}{int(x)} x {round((x-int(x))*100)}"
    a, b, diff = round(pairs.query(qs)[['a','b','dif']].max(),0)
    #return f"{size} [{a}%-{b}% ({diff}%)]"
    return f"{size} [{a}%-{b}% ({round(100*(1 - (100-b)/(100-a)),2)}% {diff}%)]"
max_trial_size = result['students_per_group'].max()+result['attempts_per_student'].max()/100
for test in tests:
    x = pd.DataFrame([{'Comparison': c, 
               **{f"{s.title()} students": unpack_trial_size(s,c,test,missing=max_trial_size) for s in ss
                  for s in ss}} for c in cs if c != 'none'])
    print(test, "Trial size: Students x attempts for each scenario [trial percent (relative and absolute change)]")
    display(x)

mlm Trial size: Students x attempts for each scenario [trial percent (relative and absolute change)]


,Comparison,Variable students,Uniform students,Initial students,Second students
0,small,> 100 x 10 [58.0%-65.0% (16.67% 7.0%)],75 x 10 [50.0%-58.0% (16.0% 8.0%)],100 x 10 [52.0%-60.0% (16.67% 8.0%)],> 100 x 10 [75.0%-79.0% (16.0% 4.0%)]
1,mod,75 x 10 [58.0%-70.0% (28.57% 12.0%)],30 x 10 [50.0%-64.0% (28.0% 14.0%)],75 x 10 [52.0%-66.0% (29.17% 14.0%)],100 x 10 [75.0%-82.0% (28.0% 7.0%)]
2,mod1,75 x 10 [58.0%-72.0% (33.33% 14.0%)],30 x 10 [50.0%-67.0% (34.0% 17.0%)],30 x 10 [52.0%-68.0% (33.33% 16.0%)],75 x 10 [75.0%-83.0% (32.0% 8.0%)]
3,big,75 x 10 [58.0%-74.0% (38.1% 16.0%)],30 x 10 [50.0%-69.0% (38.0% 19.0%)],30 x 10 [52.0%-70.0% (37.5% 18.0%)],75 x 10 [75.0%-84.0% (36.0% 9.0%)]


mlms Trial size: Students x attempts for each scenario [trial percent (relative and absolute change)]


,Comparison,Variable students,Uniform students,Initial students,Second students
0,small,> 100 x 10 [58.0%-65.0% (16.67% 7.0%)],75 x 10 [50.0%-58.0% (16.0% 8.0%)],100 x 10 [52.0%-60.0% (16.67% 8.0%)],> 100 x 10 [75.0%-79.0% (16.0% 4.0%)]
1,mod,75 x 10 [58.0%-70.0% (28.57% 12.0%)],30 x 10 [50.0%-64.0% (28.0% 14.0%)],75 x 10 [52.0%-66.0% (29.17% 14.0%)],100 x 10 [75.0%-82.0% (28.0% 7.0%)]
2,mod1,75 x 10 [58.0%-72.0% (33.33% 14.0%)],30 x 10 [50.0%-67.0% (34.0% 17.0%)],30 x 10 [52.0%-68.0% (33.33% 16.0%)],75 x 10 [75.0%-83.0% (32.0% 8.0%)]
3,big,75 x 10 [58.0%-74.0% (38.1% 16.0%)],30 x 10 [50.0%-69.0% (38.0% 19.0%)],30 x 10 [52.0%-70.0% (37.5% 18.0%)],75 x 10 [75.0%-84.0% (36.0% 9.0%)]


geeBIs Trial size: Students x attempts for each scenario [trial percent (relative and absolute change)]


,Comparison,Variable students,Uniform students,Initial students,Second students
0,small,> 100 x 10 [58.0%-65.0% (16.67% 7.0%)],75 x 10 [50.0%-58.0% (16.0% 8.0%)],100 x 10 [52.0%-60.0% (16.67% 8.0%)],> 100 x 10 [75.0%-79.0% (16.0% 4.0%)]
1,mod,75 x 10 [58.0%-70.0% (28.57% 12.0%)],30 x 10 [50.0%-64.0% (28.0% 14.0%)],30 x 10 [52.0%-66.0% (29.17% 14.0%)],100 x 10 [75.0%-82.0% (28.0% 7.0%)]
2,mod1,75 x 10 [58.0%-72.0% (33.33% 14.0%)],30 x 10 [50.0%-67.0% (34.0% 17.0%)],30 x 10 [52.0%-68.0% (33.33% 16.0%)],75 x 10 [75.0%-83.0% (32.0% 8.0%)]
3,big,30 x 10 [58.0%-74.0% (38.1% 16.0%)],30 x 10 [50.0%-69.0% (38.0% 19.0%)],30 x 10 [52.0%-70.0% (37.5% 18.0%)],75 x 10 [75.0%-84.0% (36.0% 9.0%)]


geeBI Trial size: Students x attempts for each scenario [trial percent (relative and absolute change)]


,Comparison,Variable students,Uniform students,Initial students,Second students
0,small,> 100 x 10 [58.0%-65.0% (16.67% 7.0%)],75 x 10 [50.0%-58.0% (16.0% 8.0%)],100 x 10 [52.0%-60.0% (16.67% 8.0%)],> 100 x 10 [75.0%-79.0% (16.0% 4.0%)]
1,mod,75 x 10 [58.0%-70.0% (28.57% 12.0%)],30 x 10 [50.0%-64.0% (28.0% 14.0%)],30 x 10 [52.0%-66.0% (29.17% 14.0%)],100 x 10 [75.0%-82.0% (28.0% 7.0%)]
2,mod1,75 x 10 [58.0%-72.0% (33.33% 14.0%)],30 x 10 [50.0%-67.0% (34.0% 17.0%)],30 x 10 [52.0%-68.0% (33.33% 16.0%)],75 x 10 [75.0%-83.0% (32.0% 8.0%)]
3,big,30 x 10 [58.0%-74.0% (38.1% 16.0%)],30 x 10 [50.0%-69.0% (38.0% 19.0%)],30 x 10 [52.0%-70.0% (37.5% 18.0%)],75 x 10 [75.0%-84.0% (36.0% 9.0%)]


gee Trial size: Students x attempts for each scenario [trial percent (relative and absolute change)]


,Comparison,Variable students,Uniform students,Initial students,Second students
0,small,> 100 x 10 [58.0%-65.0% (16.67% 7.0%)],75 x 10 [50.0%-58.0% (16.0% 8.0%)],100 x 10 [52.0%-60.0% (16.67% 8.0%)],> 100 x 10 [75.0%-79.0% (16.0% 4.0%)]
1,mod,75 x 10 [58.0%-70.0% (28.57% 12.0%)],30 x 10 [50.0%-64.0% (28.0% 14.0%)],75 x 10 [52.0%-66.0% (29.17% 14.0%)],100 x 10 [75.0%-82.0% (28.0% 7.0%)]
2,mod1,75 x 10 [58.0%-72.0% (33.33% 14.0%)],30 x 10 [50.0%-67.0% (34.0% 17.0%)],30 x 10 [52.0%-68.0% (33.33% 16.0%)],75 x 10 [75.0%-83.0% (32.0% 8.0%)]
3,big,30 x 10 [58.0%-74.0% (38.1% 16.0%)],30 x 10 [50.0%-69.0% (38.0% 19.0%)],30 x 10 [52.0%-70.0% (37.5% 18.0%)],75 x 10 [75.0%-84.0% (36.0% 9.0%)]


In [ ]:
print("Finding powered combinations where count of NS trials is less than 80%")
query = "comparision != 'none' and (" + ' or '.join([f"`{c}` < 20" for c in result.columns if c != 'olsns' and 'ns' in c])+")"
query = "comparision == 'small' and students_per_group in (75, 100)"
query = "comparision == 'mod' and students_per_group == 100"
print(query)
fields = ['student_set', 'comparision', 'students_per_group', 
       'attempts_per_student', 'trials',
          #'groupA', 'groupB', 'total_attempts', 'trials',
       #'geeBwrong', 'geeBns', 'geeBsig', 'geeBvsig', 'geeBp', 
       #'geeBIwrong', 'geeBIns', 'geeBIsig', 'geeBIvsig', 'geeBIp', 
       'geeBIswrong', 'geeBIsns', 'geeBIssig', 'geeBIsvsig', 'geeBIsp'
         ]
result.query(query)[fields]

In [ ]:
# Look for invalid measures:
print("These counts chould be ~2.5")
result.query("comparision == 'none'")[[c for c in result.columns if 'wrong' in c]].mean()


In [ ]:
print("Finding powered combinations where count of NS trials is less than 80%")
query = "comparision != 'none' and (" + ' or '.join([f"{c} < 20" for c in result.columns if c != 'olsns' and 'ns' in c])+")"
print("using this query:", query)
result.query(query)